## 2-6difluoropyridine Adsorption Energy Example

## You have to visualize the structures after relaxation!!

In [17]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/2_6difluoropyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [18]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_2_6difluoropyridine_5_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

$$\Delta E_{ads} = (E_{complex+H}+E_{separate ligand}) - E_{complex} - E_H$$
### We need to add the energy of the separate ligand because in the hydride complex, we remove a ligand to add the hydrogen.

In [19]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Fc1cccc(F)n1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='FC5FNH3', pbc=False)


## Compute the Energy of the Complex

In [20]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:14:50  -113965.758024       13.388062
FIRE:    1 19:14:50  -113971.107336        9.054475
FIRE:    2 19:14:51  -113974.489258        5.663538
FIRE:    3 19:14:51  -113976.622779        3.830897
FIRE:    4 19:14:51  -113978.194827        3.670660
FIRE:    5 19:14:51  -113979.564884        3.055086
FIRE:    6 19:14:51  -113980.764403        3.052824
FIRE:    7 19:14:51  -113981.798262        2.889483
FIRE:    8 19:14:51  -113982.749833        2.645628
FIRE:    9 19:14:51  -113983.636846        2.799866
FIRE:   10 19:14:51  -113984.461240        2.448217
FIRE:   11 19:14:52  -113985.284345        2.972849
FIRE:   12 19:14:52  -113986.081968        2.708284
FIRE:   13 19:14:52  -113986.734390        1.837118
FIRE:   14 19:14:52  -113987.283515        1.814190
FIRE:   15 19:14:52  -113987.856284        2.062832
FIRE:   16 19:14:52  -113988.401612        1.470583
FIRE:   17 19:14:52  -113988.917475        1.875837
FIRE:   18 19:

In [137]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 18:57:42  -101830.823185       13.485260
FIRE:    1 18:57:42  -101835.448138        8.610290
FIRE:    2 18:57:42  -101838.122473        4.981582
FIRE:    3 18:57:42  -101839.716842        3.382395
FIRE:    4 18:57:42  -101840.931463        3.319123
FIRE:    5 18:57:42  -101841.975348        3.267244
FIRE:    6 18:57:42  -101842.812074        3.110774
FIRE:    7 18:57:42  -101843.529104        2.583977
FIRE:    8 18:57:42  -101844.240367        2.083073
FIRE:    9 18:57:42  -101844.922979        2.500429
FIRE:   10 18:57:42  -101845.593076        3.465527
FIRE:   11 18:57:43  -101846.279106        2.824067
FIRE:   12 18:57:43  -101846.825277        1.630446
FIRE:   13 18:57:43  -101847.251446        2.332896
FIRE:   14 18:57:43  -101847.704506        2.268554
FIRE:   15 18:57:43  -101848.117589        1.781759
FIRE:   16 18:57:43  -101848.545320        2.068580
FIRE:   17 18:57:43  -101848.947928        1.063720
FIRE:   18 18:

In [138]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:57:49   -12158.484558        2.733934
FIRE:    1 18:57:49   -12158.649559        1.616158
FIRE:    2 18:57:49   -12158.762721        1.439178
FIRE:    3 18:57:49   -12158.819427        1.481200
FIRE:    4 18:57:50   -12158.835579        1.115155
FIRE:    5 18:57:50   -12158.854775        0.479622
FIRE:    6 18:57:50   -12158.862553        0.259000
FIRE:    7 18:57:50   -12158.862851        0.250929
FIRE:    8 18:57:50   -12158.863414        0.240537
FIRE:    9 18:57:50   -12158.864177        0.226993
FIRE:   10 18:57:50   -12158.865067        0.212585
FIRE:   11 18:57:50   -12158.866013        0.199898
FIRE:   12 18:57:50   -12158.866967        0.191012
FIRE:   13 18:57:50   -12158.867912        0.186427
FIRE:   14 18:57:51   -12158.868955        0.184537
FIRE:   15 18:57:51   -12158.870128        0.182283
FIRE:   16 18:57:51   -12158.871457        0.174586
FIRE:   17 18:57:51   -12158.872915        0.157572
FIRE:   18 18:

## Compute the energy of the Hydrogen atom

In [139]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:57:51      -31.589716        0.047594
FIRE:    1 18:57:51      -31.589744        0.012353
FIRE:    2 18:57:52      -31.589733        0.031727
FIRE:    3 18:57:52      -31.589737        0.025917
FIRE:    4 18:57:52      -31.589743        0.015339
FIRE:    5 18:57:52      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [141]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.158 eV


## 4-fluoro-2-nitrophenol

In [21]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/4fluoro2nitrophenol_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [114]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_thd_2_4fluoro2nitrophenol_3_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [115]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Oc1ccc(F)cc1[N+]([O-])=O"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OC4FC2NO2H4', pbc=False)


In [22]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:17:40  -107572.401985        6.232398
FIRE:    1 19:17:40  -107575.262534        3.128768
FIRE:    2 19:17:40  -107575.246627        6.182753
FIRE:    3 19:17:41  -107575.809057        3.244806
FIRE:    4 19:17:41  -107576.118092        0.965440
FIRE:    5 19:17:41  -107575.994704        2.596630
FIRE:    6 19:17:41  -107576.028205        2.409112
FIRE:    7 19:17:41  -107576.087021        2.039179
FIRE:    8 19:17:41  -107576.156449        1.522685
FIRE:    9 19:17:41  -107576.218712        0.995887
FIRE:   10 19:17:41  -107576.258232        0.742950
FIRE:   11 19:17:41  -107576.269754        0.746828
FIRE:   12 19:17:41  -107576.270839        0.730701
FIRE:   13 19:17:41  -107576.272963        0.716217
FIRE:   14 19:17:41  -107576.276034        0.712656
FIRE:   15 19:17:41  -107576.279938        0.707932
FIRE:   16 19:17:42  -107576.284507        0.702057
FIRE:   17 19:17:42  -107576.289610        0.695062
FIRE:   18 19:

## remove the hydrogen from the oxygen attached 

In [117]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 18:42:07   -90955.226018        6.455554
FIRE:    1 18:42:07   -90957.470190        3.114263
FIRE:    2 18:42:07   -90957.079384        8.551728
FIRE:    3 18:42:08   -90957.846291        3.931984
FIRE:    4 18:42:08   -90958.149835        1.215103
FIRE:    5 18:42:08   -90958.159015        1.205413
FIRE:    6 18:42:08   -90958.175681        1.185968
FIRE:    7 18:42:08   -90958.196934        1.156567
FIRE:    8 18:42:08   -90958.219669        1.116905
FIRE:    9 18:42:08   -90958.241620        1.066681
FIRE:   10 18:42:08   -90958.262223        1.006107
FIRE:   11 18:42:08   -90958.282578        0.936464
FIRE:   12 18:42:08   -90958.306707        0.851374
FIRE:   13 18:42:09   -90958.336219        0.750850
FIRE:   14 18:42:09   -90958.370349        0.636605
FIRE:   15 18:42:09   -90958.405272        0.688177
FIRE:   16 18:42:09   -90958.437690        0.794114
FIRE:   17 18:42:09   -90958.470378        0.876846
FIRE:   18 18:

In [77]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:25:26   -16632.678327        6.468798
FIRE:    1 18:25:27   -16633.415265        3.221794
FIRE:    2 18:25:27   -16632.925227       11.488454
FIRE:    3 18:25:27   -16633.389488        5.359196
FIRE:    4 18:25:27   -16633.571597        2.232046
FIRE:    5 18:25:27   -16633.578011        2.030837
FIRE:    6 18:25:27   -16633.589043        1.658805
FIRE:    7 18:25:27   -16633.601824        1.180648
FIRE:    8 18:25:27   -16633.613538        0.787410
FIRE:    9 18:25:27   -16633.622576        0.788035
FIRE:   10 18:25:27   -16633.629238        0.912609
FIRE:   11 18:25:28   -16633.635285        1.178325
FIRE:   12 18:25:28   -16633.643420        1.298041
FIRE:   13 18:25:28   -16633.655194        1.198807
FIRE:   14 18:25:28   -16633.670370        0.849160
FIRE:   15 18:25:28   -16633.686229        0.640120
FIRE:   16 18:25:28   -16633.699414        0.541687
FIRE:   17 18:25:28   -16633.710814        0.810681
FIRE:   18 18:

In [78]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:25:30      -31.589716        0.047594
FIRE:    1 18:25:30      -31.589744        0.012353
FIRE:    2 18:25:30      -31.589733        0.031727
FIRE:    3 18:25:30      -31.589737        0.025917
FIRE:    4 18:25:30      -31.589743        0.015339
FIRE:    5 18:25:31      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [79]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.079 eV


## Aniline

In [23]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/aniline_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [119]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_thd_2_aniline_3_hydride_1_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [120]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Nc1ccccc1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='NC6H7', pbc=False)


In [24]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:19:27   -72340.432032        1.942990
FIRE:    1 19:19:27   -72340.616893        0.618268
FIRE:    2 19:19:27   -72340.583941        1.413514
FIRE:    3 19:19:27   -72340.646344        1.068824
FIRE:    4 19:19:27   -72340.714534        0.499047
FIRE:    5 19:19:28   -72340.733831        0.620035
FIRE:    6 19:19:28   -72340.736490        0.586447
FIRE:    7 19:19:28   -72340.741414        0.522059
FIRE:    8 19:19:28   -72340.747927        0.489634
FIRE:    9 19:19:28   -72340.755303        0.475056
FIRE:   10 19:19:28   -72340.762971        0.459423
FIRE:   11 19:19:28   -72340.770742        0.443807
FIRE:   12 19:19:28   -72340.778826        0.428888
FIRE:   13 19:19:28   -72340.788644        0.413410
FIRE:   14 19:19:28   -72340.801081        0.427486
FIRE:   15 19:19:29   -72340.816792        0.399616
FIRE:   16 19:19:29   -72340.835670        0.362720
FIRE:   17 19:19:29   -72340.856754        0.345353
FIRE:   18 19:

In [122]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 18:54:59   -64531.034010        3.356951
FIRE:    1 18:54:59   -64531.305140        2.916705
FIRE:    2 18:54:59   -64531.423912        1.687140
FIRE:    3 18:54:59   -64531.481538        1.599491
FIRE:    4 18:54:59   -64531.549172        1.436584
FIRE:    5 18:54:59   -64531.580839        1.217277
FIRE:    6 18:54:59   -64531.587813        1.142769
FIRE:    7 18:54:59   -64531.594920        1.066069
FIRE:    8 18:54:59   -64531.607618        0.973247
FIRE:    9 18:54:59   -64531.623339        0.966790
FIRE:   10 18:55:00   -64531.639255        0.957149
FIRE:   11 18:55:00   -64531.653222        0.943358
FIRE:   12 18:55:00   -64531.664623        0.924428
FIRE:   13 18:55:00   -64531.674608        0.900138
FIRE:   14 18:55:00   -64531.686541        0.898941
FIRE:   15 18:55:00   -64531.703418        0.905399
FIRE:   16 18:55:00   -64531.727666        0.907124
FIRE:   17 18:55:00   -64531.759357        0.901698
FIRE:   18 18:

In [85]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:26:38    -7825.588005        1.842904
FIRE:    1 18:26:38    -7825.668573        1.131711
FIRE:    2 18:26:38    -7825.705925        0.853396
FIRE:    3 18:26:38    -7825.713616        0.725986
FIRE:    4 18:26:39    -7825.722973        0.497147
FIRE:    5 18:26:39    -7825.727929        0.350821
FIRE:    6 18:26:39    -7825.728839        0.471301
FIRE:    7 18:26:39    -7825.729549        0.443363
FIRE:    8 18:26:39    -7825.730817        0.389462
FIRE:    9 18:26:39    -7825.732379        0.313584
FIRE:   10 18:26:39    -7825.733934        0.253093
FIRE:   11 18:26:39    -7825.735229        0.220218
FIRE:   12 18:26:39    -7825.736155        0.187266
FIRE:   13 18:26:39    -7825.736796        0.195729
FIRE:   14 18:26:40    -7825.737427        0.225791
FIRE:   15 18:26:40    -7825.738303        0.256682
FIRE:   16 18:26:40    -7825.739605        0.238524
FIRE:   17 18:26:40    -7825.741250        0.189636
FIRE:   18 18:

In [86]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:26:41      -31.589716        0.047594
FIRE:    1 18:26:41      -31.589744        0.012353
FIRE:    2 18:26:41      -31.589733        0.031727
FIRE:    3 18:26:41      -31.589737        0.025917
FIRE:    4 18:26:41      -31.589743        0.015339
FIRE:    5 18:26:41      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [87]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.697 eV


# Bidentate Structures

## 2-mercaptopyridine

In [26]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/2mercaptopyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [14]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_2mercaptopyridine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [27]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "S=C1NC=CC=C1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='SCNC4H5', pbc=False)


## Compared to the ligand in the complex, this ligand has an extra hydrogen attached to the Nitrogen

In [16]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:13:42   -93767.719408        4.768582
FIRE:    1 19:13:42   -93768.915706        2.202764
FIRE:    2 19:13:42   -93769.004613        3.413841
FIRE:    3 19:13:43   -93769.178995        2.488508
FIRE:    4 19:13:43   -93769.371408        1.079424
FIRE:    5 19:13:43   -93769.444412        0.982348
FIRE:    6 19:13:43   -93769.448730        0.945330
FIRE:    7 19:13:43   -93769.456681        0.873590
FIRE:    8 19:13:43   -93769.467065        0.771902
FIRE:    9 19:13:43   -93769.478551        0.647760
FIRE:   10 19:13:43   -93769.490059        0.511776
FIRE:   11 19:13:43   -93769.501115        0.455707
FIRE:   12 19:13:43   -93769.511947        0.544407
FIRE:   13 19:13:44   -93769.524450        0.635853
FIRE:   14 19:13:44   -93769.539730        0.668133
FIRE:   15 19:13:44   -93769.558489        0.620346
FIRE:   16 19:13:44   -93769.580108        0.483413
FIRE:   17 19:13:44   -93769.602491        0.320322
FIRE:   18 19:

In [9]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:09:11   -76223.436265        5.491084
FIRE:    1 19:09:11   -76224.550947        3.115050
FIRE:    2 19:09:11   -76224.671222        3.103353
FIRE:    3 19:09:11   -76224.793824        2.245922
FIRE:    4 19:09:11   -76224.931963        0.929204
FIRE:    5 19:09:11   -76224.986765        0.625547
FIRE:    6 19:09:11   -76224.989110        0.595483
FIRE:    7 19:09:11   -76224.993441        0.593000
FIRE:    8 19:09:11   -76224.999158        0.589300
FIRE:    9 19:09:12   -76225.005588        0.584406
FIRE:   10 19:09:12   -76225.012187        0.578339
FIRE:   11 19:09:12   -76225.018718        0.571147
FIRE:   12 19:09:12   -76225.025310        0.562922
FIRE:   13 19:09:12   -76225.033073        0.552783
FIRE:   14 19:09:12   -76225.042676        0.540478
FIRE:   15 19:09:12   -76225.054594        0.525657
FIRE:   16 19:09:12   -76225.068652        0.507764
FIRE:   17 19:09:12   -76225.083865        0.485924
FIRE:   18 19:

In [10]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")
view(ligand_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:09:27   -17591.029900        3.101192
FIRE:    1 19:09:27   -17591.197802        0.951413
FIRE:    2 19:09:27   -17591.147887        2.909637
FIRE:    3 19:09:27   -17591.202807        1.954066
FIRE:    4 19:09:27   -17591.250253        0.693548
FIRE:    5 19:09:28   -17591.248242        1.056050
FIRE:    6 19:09:28   -17591.250764        0.978024
FIRE:    7 19:09:28   -17591.255171        0.827318
FIRE:    8 19:09:28   -17591.260375        0.614950
FIRE:    9 19:09:28   -17591.265164        0.466938
FIRE:   10 19:09:28   -17591.268612        0.415435
FIRE:   11 19:09:28   -17591.270509        0.369019
FIRE:   12 19:09:28   -17591.271523        0.431641
FIRE:   13 19:09:28   -17591.272828        0.550846
FIRE:   14 19:09:28   -17591.275480        0.587679
FIRE:   15 19:09:29   -17591.279974        0.502216
FIRE:   16 19:09:29   -17591.285555        0.307287
FIRE:   17 19:09:29   -17591.290164        0.234500
FIRE:   18 19:

In [11]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:09:31      -31.589716        0.047594
FIRE:    1 19:09:31      -31.589744        0.012353
FIRE:    2 19:09:31      -31.589733        0.031729
FIRE:    3 19:09:31      -31.589737        0.025916
FIRE:    4 19:09:31      -31.589743        0.015339
FIRE:    5 19:09:32      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [12]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand - E_H) - E_complex - 2*E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: 0.682 eV


In [ ]:
{"2-6difluoropyridine": 0.158,
 "4-fluoro-2-nitrophenol": -0.079,
 "aniline": 0.697,
 "2-mercaptopyridine":0.682,
}

## Adsorption energy is too high. Need to debug

## ethanolamine

In [25]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ethanolamine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [36]:
xyz_path = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/ni_oct_2_ethanolamine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
view(complex_hydride_atoms, viewer='x3d')

In [37]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "NCCO"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='NC2OH7', pbc=False)


In [46]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 18:19:25   -58208.186563        6.216393
FIRE:    1 18:19:25   -58209.734419        4.065467
FIRE:    2 18:19:25   -58210.757566        2.766711
FIRE:    3 18:19:26   -58211.373284        1.717242
FIRE:    4 18:19:26   -58211.799996        1.822356
FIRE:    5 18:19:26   -58212.097571        2.247507
FIRE:    6 18:19:26   -58212.466766        2.355979
FIRE:    7 18:19:26   -58212.648921        3.257959
FIRE:    8 18:19:26   -58212.747021        2.183644
FIRE:    9 18:19:26   -58212.854304        1.032543
FIRE:   10 18:19:26   -58212.908659        1.009361
FIRE:   11 18:19:26   -58212.933871        1.535698
FIRE:   12 18:19:26   -58212.981397        1.767795
FIRE:   13 18:19:26   -58213.069213        1.550068
FIRE:   14 18:19:26   -58213.175070        0.933124
FIRE:   15 18:19:27   -58213.264279        0.655394
FIRE:   16 18:19:27   -58213.314169        1.521219
FIRE:   17 18:19:27   -58213.382231        1.562346
FIRE:   18 18:

In [39]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:17:10   -52518.194927        9.949059
FIRE:    1 18:17:10   -52519.930410        4.800735
FIRE:    2 18:17:10   -52520.651800        2.270891
FIRE:    3 18:17:10   -52520.752632        1.797159
FIRE:    4 18:17:10   -52520.882579        1.452763
FIRE:    5 18:17:10   -52520.980721        1.271920
FIRE:    6 18:17:11   -52521.056714        1.497501
FIRE:    7 18:17:11   -52521.155541        1.680829
FIRE:    8 18:17:11   -52521.291041        1.370311
FIRE:    9 18:17:11   -52521.431004        0.855653
FIRE:   10 18:17:11   -52521.544022        0.881261
FIRE:   11 18:17:11   -52521.636354        1.108997
FIRE:   12 18:17:11   -52521.760863        0.892119
FIRE:   13 18:17:11   -52521.886705        0.598839
FIRE:   14 18:17:11   -52521.970507        0.870793
FIRE:   15 18:17:11   -52522.077229        0.767034
FIRE:   16 18:17:12   -52522.166688        0.766410
FIRE:   17 18:17:12   -52522.237631        0.967002
FIRE:   18 18:

In [40]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:17:15    -5724.616693        1.569887
FIRE:    1 18:17:15    -5724.694057        0.686646
FIRE:    2 18:17:15    -5724.685659        1.758364
FIRE:    3 18:17:15    -5724.702969        1.253375
FIRE:    4 18:17:15    -5724.721249        0.559698
FIRE:    5 18:17:15    -5724.725653        0.493929
FIRE:    6 18:17:16    -5724.726050        0.467958
FIRE:    7 18:17:16    -5724.726777        0.417746
FIRE:    8 18:17:16    -5724.727716        0.347192
FIRE:    9 18:17:16    -5724.728727        0.263508
FIRE:   10 18:17:16    -5724.729686        0.183595
FIRE:   11 18:17:16    -5724.730520        0.174287
FIRE:   12 18:17:16    -5724.731236        0.169343
FIRE:   13 18:17:16    -5724.731978        0.233598
FIRE:   14 18:17:16    -5724.732859        0.267085
FIRE:   15 18:17:16    -5724.733973        0.252544
FIRE:   16 18:17:17    -5724.735284        0.195723
FIRE:   17 18:17:17    -5724.736559        0.132401
FIRE:   18 18:

In [41]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 18:17:17      -31.589716        0.047594
FIRE:    1 18:17:18      -31.589744        0.012353
FIRE:    2 18:17:18      -31.589733        0.031727
FIRE:    3 18:17:18      -31.589737        0.025917
FIRE:    4 18:17:18      -31.589743        0.015340
FIRE:    5 18:17:18      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [42]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -14.817 eV


## Wrong Structures

In [43]:
## It has 2 fluorine atoms 
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/pyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [44]:
## It has a hydride attached
from ase.io import read
from ase.visualize import view
xyz_path_complex = '/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/HER-metal-complex-ligands/complexes/xyz/4methoxyphenol_structure.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')